In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from xgboost import plot_importance

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit

from sklearn.model_selection import cross_validate

from matplotlib import pyplot

from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

import os

In [3]:
strategyName='short'
algoName='xgb'

cv_type='kf'
#kf  or tf

#D:\JupyterCode\MyDev\AlgoTradingProject\ML_ParamTuning\data\top10
features_file='xgb_long_top10.csv'
trainData_file=os.path.abspath(r'C:\DataSets\ML_LS\ML-Long_S50M15_07To2Q20-Train_Norm.csv')

mxDepth=8

splitN=5
isShuffle=False  # if True  set random_state=xseed else None

if isShuffle==False:
    xseed=None
else:
    xseed=1990


In [4]:
#paramTuning_file=os.path.abspath(r'D:\JupyterCode\MyDev\AlgoTradingProject\ML_ParamTuning\top10\lgb_Long_top10 - test.xlsx')
paramTuning_df=pd.read_csv(features_file,usecols=['learning_rate_','n_estimators_','acc_mean_mean'])
paramTuning_df.info()

FileNotFoundError: [Errno 2] File xgb_long_top10.csv does not exist: 'xgb_long_top10.csv'

In [4]:
paramTuning_df

,learning_rate_,n_estimators_,acc_mean_mean
0,0.10,80,0.846810
1,0.10,100,0.846261
2,0.05,180,0.845637
3,0.05,100,0.846322
4,0.05,160,0.845742
5,0.05,80,0.844448
6,0.05,140,0.845505
7,0.05,200,0.845097
8,0.05,120,0.846111
9,0.10,120,0.845438


In [5]:
paramTuning=paramTuning_df.to_numpy()
paramTuning

array([[1.000000e-01, 8.000000e+01, 8.468098e-01],
       [1.000000e-01, 1.000000e+02, 8.462612e-01],
       [5.000000e-02, 1.800000e+02, 8.456372e-01],
       [5.000000e-02, 1.000000e+02, 8.463220e-01],
       [5.000000e-02, 1.600000e+02, 8.457420e-01],
       [5.000000e-02, 8.000000e+01, 8.444484e-01],
       [5.000000e-02, 1.400000e+02, 8.455052e-01],
       [5.000000e-02, 2.000000e+02, 8.450970e-01],
       [5.000000e-02, 1.200000e+02, 8.461112e-01],
       [1.000000e-01, 1.200000e+02, 8.454382e-01]])

In [6]:

df=pd.read_csv(trainData_file,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71001 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   open                    71001 non-null  float64
 1   high                    71001 non-null  float64
 2   low                     71001 non-null  float64
 3   close                   71001 non-null  float64
 4   indy_ma-550             71001 non-null  float64
 5   indy_ma-1100            71001 non-null  float64
 6   indy_hh-550             71001 non-null  float64
 7   indy_ll-550             71001 non-null  float64
 8   indy_mid-550            71001 non-null  float64
 9   indy_hh2-1100           71001 non-null  float64
 10  indy_ll2-1100           71001 non-null  float64
 11  indy_mid2-1100          71001 non-null  float64
 12  indy_macd110-440        71001 non-null  float64
 13  indy_signal110-440-110  71001 non-null  float64
 14  ind

In [7]:
select_feat=np.array(df.columns )
label=df.columns[len(df.columns)-1]
def CreateData(dfx):
    

  
 labelCol=dfx.columns[len(select_feat)-1]
 
 X_df_temp =dfx.drop(columns=['open','high','low','close',labelCol])
 Y_df_temp = dfx[labelCol]

 Xtemp=X_df_temp.to_numpy()
 ytemp=Y_df_temp.to_numpy()

 return X_df_temp,Y_df_temp, Xtemp,ytemp


In [8]:
X_df,Y_df,X,y=CreateData( df)

In [9]:
X_df.info()
#display('X_df.head(3)','X_df.tail(3)')
#Y_df.tail(5)
#X.shape
#y.shape

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71001 entries, 2007-01-03 09:45:00 to 2020-06-30 16:45:00
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   indy_ma-550             71001 non-null  float64
 1   indy_ma-1100            71001 non-null  float64
 2   indy_hh-550             71001 non-null  float64
 3   indy_ll-550             71001 non-null  float64
 4   indy_mid-550            71001 non-null  float64
 5   indy_hh2-1100           71001 non-null  float64
 6   indy_ll2-1100           71001 non-null  float64
 7   indy_mid2-1100          71001 non-null  float64
 8   indy_macd110-440        71001 non-null  float64
 9   indy_signal110-440-110  71001 non-null  float64
 10  indy_hist_macd110-440   71001 non-null  float64
 11  indy_rsi25-ma20         71001 non-null  float64
 12  indy_6ATRTrail_DC-110   71001 non-null  float64
 13  cate_3trend-550_ma110   71001 non-null  int64  
 14  cat

In [10]:
Y_df.value_counts()

0    41396
1    29605
Name: LongSignal, dtype: int64

In [11]:
def CV_XFold(model,X,y):
 
 if cv_type=='tf':
  kfold =  TimeSeriesSplit(n_splits=splitN,max_train_size=None)
 elif cv_type=='kf':
  kfold = KFold(n_splits=splitN, shuffle=isShuffle, random_state=xseed)
 
 cv_scores = cross_validate(model, X, y, cv=kfold, scoring = ['accuracy'], return_estimator =True)
 
 return cv_scores

In [12]:
'''
model = XGBClassifier(n_estimators=100,max_depth=mxDepth,learning_rate=0.05,importance_type='gain')
cv_scores=CV_TimeSeriesSplit(model,X,y)
mean_score=np.mean(cv_scores['test_accuracy'])  
print(mean_score)
'''

"\nmodel = XGBClassifier(n_estimators=100,max_depth=mxDepth,learning_rate=0.05,importance_type='gain')\ncv_scores=CV_TimeSeriesSplit(model,X,y)\nmean_score=np.mean(cv_scores['test_accuracy'])  \nprint(mean_score)\n"

In [13]:

count=0;
list_dfx=[]
for param in paramTuning:
  
 count+=1
    
 learnRate=param[0]
 nTree=int(param[1])  
 #nLeave=int(param[2]) # lgm  
 #acc=param[2] 
 
 model = XGBClassifier(n_estimators=nTree,max_depth=mxDepth,learning_rate=learnRate,importance_type='gain')

 cv_scores=CV_XFold(model,X,y)
 mean_score=np.mean(cv_scores['test_accuracy'])  
 #print(learnRate,nTree,acc,mean_cv)   

 feature_importancesList=[]
 for idx,estimator in enumerate(cv_scores['estimator']):
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X_df.columns,
                                        columns=['importance'])#.sort_values('importance', ascending=False)
    feature_importancesList.append(feature_importances)

 #create column by no. split
 for i in range(splitN):
    feature_importancesList[i].rename(columns={'importance':f'impFeat{(i+1)}'},inplace=True)
 
 df_temp=pd.concat(feature_importancesList,axis=1)
    
 sum_col=f'SumFeat_{count}_{np.round(mean_score,3)}'
 df_temp[sum_col]=df_temp.sum(axis=1)
 df_features=df_temp[[sum_col]]
    

 list_dfx.append(df_features)
    
  
 #print('+++++++++++++++++++++++++++++++++++++++++++++++')
 #print(df_features)
    

In [15]:
df = pd.concat(list_dfx,axis=1)
df['TotalFeat']=df.sum(axis=1)
df['AvgFeat']=df.mean(axis=1)
df

,SumFeat_1_0.842,SumFeat_2_0.841,SumFeat_3_0.843,SumFeat_4_0.846,SumFeat_5_0.844,SumFeat_6_0.846,SumFeat_7_0.843,SumFeat_8_0.843,SumFeat_9_0.845,SumFeat_10_0.841,TotalFeat,AvgFeat
indy_ma-550,0.408652,0.416759,0.395721,0.384116,0.391125,0.381764,0.389502,0.395921,0.386373,0.418551,3.968484,0.721543
indy_ma-1100,2.436311,2.427725,2.474590,2.495172,2.465261,2.551923,2.473603,2.486367,2.477924,2.447759,24.736635,4.497570
indy_hh-550,0.275550,0.283679,0.275128,0.256207,0.271929,0.245017,0.268768,0.277521,0.265670,0.287649,2.707118,0.492203
indy_ll-550,0.107327,0.109352,0.105798,0.100688,0.105326,0.097016,0.104479,0.105518,0.103168,0.108438,1.047110,0.190384
indy_mid-550,0.155607,0.157916,0.155064,0.145854,0.153435,0.139751,0.151213,0.156492,0.149465,0.158198,1.522993,0.276908
indy_hh2-1100,0.383427,0.385006,0.378030,0.405283,0.383042,0.402916,0.386261,0.375297,0.396194,0.383683,3.879138,0.705298
indy_ll2-1100,0.127771,0.125464,0.125040,0.127190,0.125712,0.123586,0.125639,0.124126,0.126557,0.123147,1.254231,0.228042
indy_mid2-1100,0.145053,0.143412,0.147616,0.148906,0.148289,0.147081,0.148405,0.146663,0.148580,0.142177,1.466182,0.266578
indy_macd110-440,0.172721,0.172406,0.170780,0.170067,0.170937,0.167109,0.170702,0.169851,0.171486,0.167692,1.703750,0.309773
indy_signal110-440-110,0.114122,0.112248,0.109826,0.111122,0.110952,0.109005,0.111618,0.108232,0.112370,0.107738,1.107233,0.201315


In [16]:
#D:\JupyterCode\MyDev\AlgoTradingProject\ML_ParamTuning\FindFeat_CV\feat_result
df.to_csv(f'{strategyName}_{algoName}_{cv_type}_split{splitN}.csv')
    

# Merge XF 

In [88]:
import pandas as pd
import numpy as np

In [90]:
list_files=['feat_result\short_xgb_kf_split5.csv','feat_result\short_xgb_ts_split5.csv']
list_df=[]

for file in list_files:
 df_temp=pd.read_csv(file)
 #df_temp=df_temp.iloc[:,[0,len(df_temp.columns)-1,len(df_temp.columns)-2]]
 df_temp=df_temp.iloc[:,[0,len(df_temp.columns)-2]]
 list_df.append(df_temp)
 
dfx=pd.concat(list_df,axis=0)
dfx.reset_index(drop=True,inplace=True)
dfx.rename(columns={'Unnamed: 0':'Feature'},inplace=True)
dfx

,Feature,TotalFeat
0,indy_ma-550,4.048381
1,indy_ma-1100,24.976269
2,indy_hh-550,2.819970
3,indy_ll-550,1.052441
4,indy_mid-550,1.555718
5,indy_hh2-1100,3.735552
6,indy_ll2-1100,1.210822
7,indy_mid2-1100,1.446428
8,indy_macd110-440,1.665098
9,indy_signal110-440-110,1.056564


In [91]:
featGroup_df=dfx.groupby(['Feature'],as_index=False).agg([np.sum])
featGroup_df.columns = ["_".join(x) for x in featGroup_df.columns.ravel()]
featGroup_df.sort_values(by='TotalFeat_sum',ascending=False,inplace=True)

In [92]:
featGroup_df.to_excel(f'feat_best\\best_{algoName}_{strategyName}.xlsx')